In [1]:
import pandas as pd
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [2]:

import NeuralNetwork.pretrained_models as pre_models
from NeuralNetwork.tools import predict, predict_mul
from NeuralNetwork.preprocessing import skewness_pure, min_max_normalization_pure


def predict_df(X_df):

    model_A3_015, input_features_A3_015 = pre_models.Input_A3_015('weights/')
    
    outputs = ['Input_A3_015']

    preprocessings = ['min_max_normalization']

    models = [model_A3_015]

    input_features = [input_features_A3_015]

    result = {}

    for i in range(len(outputs)):

            if preprocessings[i] == 'skewness':
                X = skewness_pure(X_df, input_features[i])
            elif preprocessings[i] == 'min_max_normalization':
                X = min_max_normalization_pure(X_df, input_features[i])

            result[outputs[i]] = predict_mul(models[i], X)
    
    result_df = pd.DataFrame(result, columns = outputs)
    return result_df


In [3]:
def nn_ave(x, y):

    rmse = []
    r2 = []
    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(
            x, y, test_size=0.2, random_state=i)

        nn_pred = predict_df(X_test)

        rmse.append(math.sqrt(mean_squared_error(y_test, nn_pred)))
        r2.append(r2_score(y_test, nn_pred))

    test_rmse = np.array(rmse).mean()
    print('test_rmse_ave:', test_rmse)
    print(rmse)
    print('\n')
    test_r2 = np.array(r2).mean()
    print('test_r2_ave:', test_r2)
    print(r2)

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [5]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [6]:
fea_number=7
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A3_015.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
nn=data.drop(['Predict'], axis=1)
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A2_015', 'Input_A6_015', 'Input_A1_015', 'Input_A4_015',
       'Input_A5_015', 'Input_C_137', 'Input_A4_012'], dtype=object)

In [7]:
test=pd.read_csv('./final_testX.csv',index_col=0).drop(['Number'], axis=1)[title] ## 改路徑

In [8]:
nn_ave(nn, y)

test_rmse_ave: 0.024578226914792382
[0.02521454114422555, 0.02616802301348055, 0.023194927770656338, 0.0214952167039765, 0.025185665354734543, 0.02445537792173391, 0.024248494454938593, 0.025343751437680133, 0.021190402619349683, 0.02928586872714807]


test_r2_ave: 0.34846017953047975
[0.34655296366521526, 0.3056084760863683, 0.41240588771437925, 0.47544040723495706, 0.309401610542836, 0.2835298622329553, 0.44117527506085774, 0.3769320265796403, 0.5937197092639594, -0.06016442307637093]


## Linear Regression Baseline

In [4]:
lr_rmse_ave_fea(skew_data,7)

train_rmse: 0.019713512282696334
test_rmse: 0.020504559214319984
test_r2: 0.5508460551254092


## Lasso

In [5]:
laso_rmse_ave(skew_data[title],y,9e-05)

test_rmse_ave: 0.020376659127956644
[0.019595678141870715, 0.025034946710217745, 0.019748459743782903, 0.01722057450847281, 0.020191466306880998, 0.021051460347164982, 0.02180145803923424, 0.019527462811933516, 0.02008410499501116, 0.01951097967499735]


test_r2_ave: 0.5575229992804258
[0.6053347859044851, 0.36444092037359566, 0.5740507629884186, 0.663328425348323, 0.5561314020965205, 0.4690986943095694, 0.5482718725913468, 0.6300992037017203, 0.6350340872484874, 0.5294398382417908]


## ElasticNet

In [6]:
ElasticNet_rmse_ave(skew_data[title],y,[0.02],0.015)

test_rmse_ave: 0.020383014180187543
[0.019430199078172623, 0.024928376107396625, 0.0197436588685917, 0.01727244446105894, 0.020326660624560632, 0.02100937256195339, 0.021813599767353902, 0.01967778643424961, 0.020049446569995463, 0.019578597328542547]


test_r2_ave: 0.5573370734154943
[0.611972277480483, 0.369840392713519, 0.5742578353969808, 0.6612971993988458, 0.550167554962861, 0.47121941404366585, 0.5477685771336773, 0.624382244684153, 0.6362926177764856, 0.5261726205642723]


## SVR

In [7]:
model=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,7,model)

train_rmse: 0.020459710392575232
test_rmse: 0.021243930263697586
test_r2: 0.5175633683687253


## XGB

In [8]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.1,
              booster = 'dart', 
              n_estimators = 80, 
              max_depth = 4, 
              min_child_weight = 2,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.6,
              reg_alpha =  0,
              reg_lambda = 0.5)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.019314022878319733
[0.018708557406221512, 0.022736903489053748, 0.016826896137625908, 0.020037382268678624, 0.019020752525853828, 0.019519288363624935, 0.018230421629519142, 0.019578933496952357, 0.019179014945796907, 0.01930207851987036]


test_r2_ave: 0.6019573658397293
[0.6402598960205762, 0.47576595508667463, 0.6907573296716079, 0.5441802431251226, 0.6061107807272861, 0.5435667261781567, 0.6841364728605484, 0.6281466572582409, 0.6671872674572534, 0.5394623300118255]


## RandomForest

In [9]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 9,
            min_samples_split = 7,
            max_leaf_nodes = 14,
            min_samples_leaf =4,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.01900400321155791
[0.017769769914259874, 0.02212374567080392, 0.016477233385604258, 0.01862246715672049, 0.019188050881242875, 0.018992173620575408, 0.018509244524834073, 0.01962633862027239, 0.019840737294604338, 0.01889027104666146]


test_r2_ave: 0.6159384823977577
[0.6754572863688693, 0.503659287912025, 0.7034759160791153, 0.6062816903429231, 0.5991513484533801, 0.5678856585919713, 0.6744007165937668, 0.6263437915398141, 0.6438253950419099, 0.5589037330538018]


In [9]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=9e-05)
elastic_mod=ElasticNet(alpha=[0.02], l1_ratio=0.015)
svr_mod=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.1,
              booster = 'dart', 
              n_estimators = 80, 
              max_depth = 4, 
              min_child_weight = 2,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.6,
              reg_alpha =  0,
              reg_lambda = 0.5)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 9,
            min_samples_split = 7,
            max_leaf_nodes = 14,
            min_samples_leaf =4,
            random_state = 42)     
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('Elastic', elastic_mod),
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])

# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.01908822294563444
[0.017962630120056063, 0.023184440634087086, 0.017599891275272977, 0.01752531451396, 0.018956884105527667, 0.019719143457516602, 0.01927516182678679, 0.018915986132989622, 0.019005097104266233, 0.018737680285881346]


test_r2_ave: 0.611821945775781
[0.6683743544507375, 0.45492554486446535, 0.6616927098587568, 0.6513073290252465, 0.6087515643244157, 0.5341721714447243, 0.6468964202868752, 0.6529024075149485, 0.6731958795890033, 0.5660010763986368]


In [10]:

stack_mod = StackingRegressor(regressors=[lasso_mod,elastic_mod,  vote_mod], 
                           meta_regressor=random_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.020764845095189843
[0.02016537518443068, 0.02382798067406965, 0.016076886899909583, 0.02148947673001909, 0.021241837450419025, 0.02148471063517267, 0.01920625714889707, 0.021231926695636862, 0.02187611159312292, 0.0210478879402209]


test_r2_ave: 0.538701132406507
[0.5820533318995379, 0.4242458682510616, 0.7177101261514456, 0.4757205213112644, 0.5087496572093333, 0.447021362249715, 0.649416451032429, 0.5627070863762103, 0.5670002975279504, 0.45238662205612146]


In [11]:
ave(skew_data[title],y,0.2,stack_mod,0.4,vote_mod,0.4,random_mod)

test_rmse_ave: 0.019034248036129953
[0.017990439578783275, 0.0225470703361079, 0.016466691535045888, 0.018465322014602572, 0.019060931714124527, 0.019509409791606643, 0.01858763335065617, 0.019177019310881668, 0.019565419011582014, 0.018972543717908887]


test_r2_ave: 0.614064557633595
[0.6673467244685685, 0.48448319857086464, 0.7038552167243105, 0.6128984153168584, 0.6044449302776408, 0.5440286044925454, 0.6716369741120757, 0.6432566891598801, 0.653641664376422, 0.5550531588367856]


In [12]:
name='Input_A3_015'
test_pred=[]
seed=[0,2,3,9,6,5,8]
stack_w=0.2
vote_w=0.4
model_w=0.4
model=random_mod
for i in seed:
    X_train, X_test, y_train, y_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)

    stack = stack_mod.fit(X_train, y_train)
    stack_pred = stack.predict(test)
    vote = vote_mod.fit(X_train, y_train)
    vote_pred = vote.predict(test)
    mod = model.fit(X_train, y_train)
    mod_pred = mod.predict(test)

    ###
    final_test = (vote_w*vote_pred+stack_w*stack_pred + model_w*mod_pred)
    ###
    test_pred.append(final_test)



final_test = pd.DataFrame(np.mean(test_pred, axis=0), columns=[name])
final_test.to_csv(('./output/'+name+'.csv'))